Import  Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score,train_test_split
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.ensemble import  RandomForestRegressor
# from xgboost.sklearn import XGBRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.linear_model import LinearRegression  
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import ExtraTreesRegressor
from lightgbm import LGBMRegressor

In [3]:
df = pd.read_csv("data/autos.csv", header=0, sep=',', encoding= 'Latin1')

C:\Users\siddh\AppData\Local\Temp\ipykernel_6768\4279674029.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/autos.csv", header=0, sep=',', encoding= 'Latin1')


In [4]:
print(df.seller.value_counts())
df[df.seller != 'gewerblich']
df = df.drop('seller',1)


privat        371534
gewerblich         3
golf               1
Name: seller, dtype: int64


C:\Users\siddh\AppData\Local\Temp\ipykernel_6768\3080827631.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('seller',1)


In [5]:

print(df.offerType.value_counts ())
df[df.offerType != 'Gesuch']
df=df.drop('offerType',1)



C:\Users\siddh\AppData\Local\Temp\ipykernel_6768\390018200.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop('offerType',1)


Angebot    371525
Gesuch         12
150000          1
Name: offerType, dtype: int64


In [6]:
df= df[ (df. powerPS> 50) & (df. powerPS < 900)]
print (df. shape)
df= df[ (df. yearOfRegistration> 1950) & (df. yearOfRegistration < 2017)]
print(df.shape)

(319717, 18)
(309173, 18)


In [7]:
df.drop(['abtest','nrOfPictures','lastSeen','postalCode','dateCreated'],axis = 'columns', inplace=True)

In [8]:
new_df = df.copy()
new_df = new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])

In [9]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'), inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'), inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),
                            ('small car','convertible','combination','others'), inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'), inplace=True)

In [10]:
new_df = new_df[(new_df.price>=100) & (new_df.price<=150000)]
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df['gearbox'].fillna(value='not-declared',inplace=True)
new_df['vehicleType'].fillna(value='not-declared',inplace=True)
new_df['model'].fillna(value='not-declared',inplace=True)

In [11]:
new_df.to_csv("autos_preprocessed.csv")

Label Encoding

In [12]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(new_df[i])
    tr = mapper[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    new_df.loc[:, i+'_labels'] = pd.Series(tr, index=new_df.index)
    
labeled = new_df[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration']
                  +[x+"_labels" for x in labels]]

print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


Splitting Data Into Independent And Dependent Variables

In [13]:
X = labeled.iloc[:,1:].values
Y = labeled.iloc[:,0].values.reshape(-1,1)

In [14]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [15]:
regressor = LGBMRegressor(boosting_type="gbdt",learning_rate=0.07,metric="rmse",n_estimators=300,objective="root_mean_squared_error",random_state=42,reg_sqrt=True)
regressor.fit(X_train, Y_train)
X_train.shape

d:\setup\anaconda\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(169156, 10)

In [16]:
y_pred = regressor.predict (X_test)
print(r2_score (Y_test, y_pred))
filename ='resale_model. sav'
pickle.dump(regressor, open (filename, 'wb'))

0.8638584833681153
